In [1]:
from lltk.imports import *
C=load_corpus('LongArcPrestige')

In [8]:
dtm=C.dtm(n=25000, texts=C.meta.query('genre=="Poetry"'), meta=['reviewed'])
dtm = to_counts(dtm, scale_by=100000)
dtm

Mapping rescale_col() [x7]: 100%|██████████| 19070/19070 [00:05<00:00, 3345.76it/s]


the   and    of    to     a    in  \
reviewed   id                                                              
Reviewed   dul1.ark+=13960=t5fb5xg2z  4726  3723  2524  1204   910  1257   
           dul1.ark+=13960=t75t4h116  5158  3099  2010  1510   957  1379   
           dul1.ark+=13960=t75t4h116  5158  3099  2010  1510   957  1379   
           dul1.ark+=13960=t75t4h116  5158  3099  2010  1510   957  1379   
           dul1.ark+=13960=t75t4h116  5158  3099  2010  1510   957  1379   
...                                    ...   ...   ...   ...   ...   ...   
Unreviewed wu.89099777773             5773  3716  2037  1723  2179  1196   
           wu.89099777773             5773  3716  2037  1723  2179  1196   
           wu.89099777773             5773  3716  2037  1723  2179  1196   
           wu.89099777773             5773  3716  2037  1723  2179  1196   
           wu.89099921512             4855  3007  3034  1808  1618  1896   

                                      that  with  his   is  ...  generously  \
reviewed   id                                               ...               
Reviewed   dul1.ark+=13960=t5fb5xg2z   400   910  496  348  ...           0   
           dul1.ark+=13960=t75t4h116   657   840  604  640  ...           0   
           dul1.ark+=13960=t75t4h116   657   840  604  640  ...           0   
           dul1.ark+=13960=t75t4h116   657   840  604  640  ...           0   
           dul1.ark+=13960=t75t4h116   657   840  604  640  ...           0   
...                                    ...   ...  ...  ...  ...         ...   
Unreviewed wu.89099777773              661   582  532  736  ...           0   
           wu.89099777773              661   582  532  736  ...           0   
           wu.89099777773              661   582  532  736  ...           0   
           wu.89099777773              661   582  532  736  ...           0   
           wu.89099921512              561   559  775  406  ...           0   

                                      siamese  pendulum  organise  dejectedly  \
reviewed   id                                                                   
Reviewed   dul1.ark+=13960=t5fb5xg2z        0         0         0           0   
           dul1.ark+=13960=t75t4h116        0         0         0           0   
           dul1.ark+=13960=t75t4h116        0         0         0           0   
           dul1.ark+=13960=t75t4h116        0         0         0           0   
           dul1.ark+=13960=t75t4h116        0         0         0           0   
...                                       ...       ...       ...         ...   
Unreviewed wu.89099777773                   0         0         0           0   
           wu.89099777773                   0         0         0           0   
           wu.89099777773                   0         0         0           0   
           wu.89099777773                   0         0         0           0   
           wu.89099921512                   0         0         0           0   

                                      vulnerable  reefers  fortresses  immune  \
reviewed   id                                                                   
Reviewed   dul1.ark+=13960=t5fb5xg2z           0        0           0       0   
           dul1.ark+=13960=t75t4h116           0        0           0       0   
           dul1.ark+=13960=t75t4h116           0        0           0       0   
           dul1.ark+=13960=t75t4h116           0        0           0       0   
           dul1.ark+=13960=t75t4h116           0        0           0       0   
...                                          ...      ...         ...     ...   
Unreviewed wu.89099777773                      0        0           2       0   
           wu.89099777773                      0        0           2       0   
           wu.89099777773                      0        0           2       0   
           wu.89099777773                      0        0           2       0

,Reviewed,Unreviewed
word,,
the,5191.0,5005.0
and,3246.0,2970.0
of,2108.0,2185.0
to,1533.0,1680.0
a,1455.0,1317.0
...,...,...
vulnerable,0.0,0.0
reefers,0.0,0.0
fortresses,0.0,0.0


In [ ]:
dtm_counts

In [ ]:
gby=['reviewed']
dtmq=dtm_counts.join(C.meta[gby],lsuffix='_w')
dtmq=dtmq.reset_index().set_index(gby + [dtm.index.name])
dtmq

In [27]:
def do_mannwhitney(obj):
    from scipy.stats import mannwhitneyu,zscore
    
    x,y,meta=obj
    xz=zscore(x)
    yz=zscore(y)
    mwU, pvalue = mannwhitneyu(x,y)
    result_dict=dict(meta.items())
    result_dict['U']=mwU
    result_dict['pvalue']=pvalue
    result_dict['is_signif']=issign=pvalue<=0.05
    result_dict['group1_mean']=xm=x.mean()
    result_dict['group2_mean']=ym=y.mean()
    result_dict['distinctive_of']=None if not issign else (
        meta.get('group1') if xm>ym else meta.get('group2')
    )
    return result_dict


def to_mdw_mannwhitney(dtm,groupby,words=set(),group_names=None,**pmap_kwargs):
    # get words
    groups = dtm.groupby(groupby)
    if not words:
        for gname,gdf in groups:
            words = set(gdf.columns) if not words else words&set(gdf.columns)

    objs=[]
    for group1name,group1df in groups:
        for group2name,group2df in groups:
            if group1name>=group2name: continue
            for word in words:
                x=group1df[word]
                y=group2df[word]
                meta={
                    'group1':group1name,
                    'group2':group2name,
                    'word':word,
                }
                objs+=[(x,y,meta)]
    odf = pd.DataFrame(
        pmap(
            do_mannwhitney,
            objs,
            **pmap_kwargs
        )
    ).sort_values('U')
    
    return odf.set_index(['distinctive_of','group1','group2','word'])

In [29]:
mdw=to_mdw_mannwhitney(dtm, 'reviewed', num_proc=1)
mdw

Mapping do_mannwhitney() [x1]: 100%|██████████| 19070/19070 [00:11<00:00, 1703.16it/s]


U        pvalue  \
distinctive_of group1   group2     word                                 
Unreviewed     Reviewed Unreviewed oft         612051.5  2.405802e-51   
Reviewed       Reviewed Unreviewed blown       630325.5  1.962428e-49   
                                   black       664944.5  1.391509e-35   
Unreviewed     Reviewed Unreviewed reign       667942.0  5.439110e-37   
Reviewed       Reviewed Unreviewed thin        668256.5  6.744218e-38   
...                                                 ...           ...   
NaN            Reviewed Unreviewed psalter     915830.0  4.993123e-01   
                                   ancients    915830.5  4.996184e-01   
                                   bitterest   915835.5  4.998766e-01   
                                   bequeathed  915837.0  4.998933e-01   
                                   willful     915838.0  4.998971e-01   

                                               is_signif  group1_mean  \
distinctive_of group1   group2     word                                 
Unreviewed     Reviewed Unreviewed oft              True    14.721094   
Reviewed       Reviewed Unreviewed blown            True     9.497656   
                                   black            True    29.211719   
Unreviewed     Reviewed Unreviewed reign            True     4.210938   
Reviewed       Reviewed Unreviewed thin             True     8.141406   
...                                                  ...          ...   
NaN            Reviewed Unreviewed psalter         False     0.142187   
                                   ancients        False     0.267969   
                                   bitterest       False     0.637500   
                                   bequeathed      False     0.297656   
                                   willful         False     0.095312   

                                               group2_mean  
distinctive_of group1   group2     word                     
Unreviewed     Reviewed Unreviewed oft           32.647100  
Reviewed       Reviewed Unreviewed blown          4.730957  
                                   black         16.402516  
Unreviewed     Reviewed Unreviewed reign         10.102027  
Reviewed       Reviewed Unreviewed thin           4.466806  
...                                                    ...  
NaN            Reviewed Unreviewed psalter        0.083857  
                                   ancients       0.450035  
                                   bitterest      0.807128  
                                   bequeathed     1.306778  
                                   willful        0.122991  

[19070 rows x 5 columns]

In [ ]:

# def compute_mannwhitney(group1,group2,words=None):
#     from scipy.stats import mannwhitneyu,zscore
#     from tqdm import tqdm
#     import pandas as pd

#     if not words:
#         words = set(group1.columns) & set(group2.columns)
    
#     result_list=[]
#     for word in tqdm(words):
#         x=group1[word]
#         y=group2[word]

#         xz=zscore(group1[word])
#         yz=zscore(group2[word])
        
#         mwU, pvalue = mannwhitneyu(x,y)
    
#         result_dict={}
#         result_dict['word']=word
#         result_dict['U']=mwU
#         result_dict['pvalue']=pvalue
#         result_dict['is_signif']=pvalue<=0.05
#         result_dict['group1_mean']=x.mean()
#         result_dict['group2_mean']=y.mean()
#         result_dict['group1_mean_z']=xz.mean()
#         result_dict['group2_mean_z']=yz.mean()
#         result_list.append(result_dict)
        
#     return pd.DataFrame(result_list).sort_values('U')

In [ ]:
dfres=compute_mannwhitney(dtmq.loc['Reviewed'], dtmq.loc['Unreviewed'])
dfres

In [ ]:
dfres.head(200).sort_values('group2_mean_z').head(20)